## Manuscript Keyword Search Program

In [ ]:
# Note: may be some way to use this to get the token offset to get an appximate page number: https://www.nltk.org/api/nltk.text.html#module-nltk.text

In [1]:
%cd /Users/lisamcharron/Desktop/SMR_PythonProgram

/Users/lisamcharron/Desktop/SMR_PythonProgram


In [2]:
#import necessary packages
from pathlib import Path
from tkinter.filedialog import askopenfilenames
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.text import Text, TokenSearcher
import re
from pdfminer.high_level import extract_text, extract_pages
from IPython.core.display import display, HTML

/var/folders/sb/xyqjv55n48d4xs54qsj7_m0c0000gn/T/ipykernel_42750/2204756225.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Define functions

In [3]:
def all_pdfs_in_folder(folder_path):
    return Path(folder_path).glob("*pdf")

def context_search_string(keyword):
    return f"<.*>{{0,50}}<{keyword}.*><.*>{{0,50}}" 

def find_contexts(text, keyword):
    search_formatted = context_search_string(keyword)
    tokens = wordpunct_tokenize(text)
    textList = Text(tokens)
    searcher = TokenSearcher(textList)
    contexts = searcher.findall(search_formatted)
    contexts = [" ".join(c) for c in contexts]
    return contexts

def count_pages(pdf_path):
    page_count = len(list(extract_pages(pdf_path)))
    return page_count

def estimate_page_number(text, context, page_count):
    # location = text.find(context) / len(text)
    # approx_page = page_count*location
    # return approx_page
    return text.find(context)

def html_highlight_by_keyword(keyword, context):
    span_color = "<span style='color:red'>{colored_text}</span>"
    pattern = f"({keyword}\w*)"
    replace = span_color.format(colored_text=r"\1")
    return re.sub(pattern, replace, context, flags=re.IGNORECASE)

def html_matches_contexts_table(keywords_and_contexts, pages, title):
    table_header = f"""
    <link rel="stylesheet" href="https://cdn.simplecss.org/simple.min.css">
    <h1>{title}</h1>
    <table>
        <tr>
            <th>Keyword</th>
            <th>Approx. Page</th>
            <th>Context</th>
        </tr>
    """
    table_row = """
        <tr>
            <td>{keyword}</td>
            <td>{page}</td>
            <td>{context}</td>
        </tr>
    """
    table_rows = [table_row.format(keyword=k, page=p, context=c) for (k, c), p in zip(keywords_and_contexts, pages)]
    return table_header + "".join(table_rows) + "</table>"

def to_html_file(file_path, html_string):
    with open(file_path, "w") as f:
        f.write(html_string)

### Get file paths to iterate through

In [4]:
pdf_folder = Path("./manuscripts")
pdf_paths = [pdf_path for pdf_path in all_pdfs_in_folder(pdf_folder)]
pdf_paths

[PosixPath('manuscripts/SMR-23-0074_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0073_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0072_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0075_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0078_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0079_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0077_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0070_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0071_Proof_hi.pdf'),
 PosixPath('manuscripts/SMR-23-0076_Proof_hi.pdf')]

### Input keywords

In [5]:
keywords = ["author", "blind", "mask", "redact", "remov", "omit", "xxx", "anon", "replicat", "repro", "repos", "code", "syntax", " script", "package", "program", "osf", "figshare", "github", "cran", "stata", "python", "availab"]

### Run program

In [6]:
tables = []
for pdf_path in pdf_paths:
    page_count = count_pages(pdf_path)
    text = extract_text(pdf_path)
    text = text.lower()
    keywords_and_contexts = []
    for keyword in keywords:
        contexts = find_contexts(text, keyword)
        for context in contexts:
            context_highlighted = html_highlight_by_keyword(keyword, context)
            keywords_and_contexts.append([keyword, context_highlighted])
    title = pdf_path.stem
    pages = [estimate_page_number(text, c, page_count) for k, c in keywords_and_contexts]
    table = html_matches_contexts_table(keywords_and_contexts, pages, title)
    tables.append(table)
html = "".join(tables)
to_html_file("./index.html", html)
display(HTML(html))          

Keyword,Approx. Page,Context
remov,-1,"53 54 55 56 57 58 59 60 o r p accordingly . for sampling purposes , however , it is not necessary for researchers to search the sampling frame for inactive or miscategorized cases . instead , if researchers discover an ineligible case among their sample , they can remove it from the sample and replace it with a new case drawn at random from the sampling frame ( see harter et al ., 2010 : 173 - 4 ). alternatively , researchers could address overcoverage by oversampling ( kalton , 2021 ). the second limitation concerns duplication error :"
remov,-1,"the past year , and if so , request the address of its previous location . then , researchers can search the sampling frame for duplicate cases of those organizations . if they discover a duplicate in the sampling frame , they can make the population count more accurate by removing the duplicate ; specifically , the case https :// mc . manuscriptcentral . com / smr sociological methods and research page 16 of 45 with the less - current location information . they can also correct the duplicate case ’ s doubled likelihood of being selected by applying a weight"
remov,-1,"from the sampling frames it produces , researchers can draw multiple samples of varying sizes without f o r p increasing cost . in addition , because the method allows researchers to specify the geographic area and search criteria , researchers can save time by not having to identify and remove organizations outside their scope of study . e e furthermore , the information that traditional sampling frame sources contain is often outdated because they are typically updated and published no more than once per year . in contrast , because r the information on google maps is continuously updated ,"
omit,-1,"concerns undercoverage error : not including organizations that are in the target population ( eligible cases not included in the sampling frame ). the types of eligible but f o r p erroneously excluded cases include 1 ) organizations that are not on the text layer of google maps ( omitted cases ) and 2 ) organizations that google maps miscategorized as not being the target population type . for example , the congregation sampling frame we generated does not include a e e particular congregation because google maps miscategorized it as a food bank . our analysis indicates that google"
omit,-1,"kobayashi et al ., 2018 ), and because it allows users to correct organizations ’ information on google maps . undercoverage will create an undercount of the target population . however , researchers can conduct analyses similar to those discussed above to estimate the percentage of eligible organizations that were omitted or miscategorized , and then revise the estimated target population size accordingly . in addition , google maps ’ very low undercoverage error and our analysis of its representativeness https :// mc . manuscriptcentral . com / smr 1 2 3 4 5 6 7 8 9 10 11 12"
repos,-1,"and sample sources possess significant limitations that undermine their coverage , representativeness , and utility . to address these challenges , we developed a new method for generating organizational sampling frames that compiles publicly w available data contained on the google maps platform . google maps functions as a data repository that gathers information from multiple sources and continually organizes and updates its data . more specifically , google maps ’ “ places ” feature obtains and curates information about every organization on its platform . 2 the program we developed extracts this information to generate 1 the probability that an"
repos,-1,"the limitations associated with using google maps to generate sampling frames are minimal , especially compared to the limitations of alternative methods , and can be mitigated through supplemental analyses . in addition , sampling frames generated by this approach can be enhanced by applying it to other online data repositories that obtain their data f